In [2]:
import tensorflow as tf

In [3]:
x1= tf.constant(5)

x2= tf.constant(6)

result= tf.multiply(x1, x2)
print(result)

with tf.Session() as sess:
    output= sess.run(result)
    print(output)
    
print(output)

Tensor("Mul:0", shape=(), dtype=int32)
30
30


In [4]:
from tensorflow.examples.tutorials.mnist import input_data

mnist= input_data.read_data_sets('/tmp/data', one_hot=True)

n_nodes_hl1= 500
n_nodes_hl2= 250
n_nodes_hl3= 100

n_classes=10
batch_size=100
#height x width
x=tf.placeholder('float', [None, 784])
y=tf.placeholder('float')

def neural_network_model(data):
    
    hidden_1_layer= {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    
    output_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1= tf.add(tf.matmul(data, hidden_1_layer['weights']) , hidden_1_layer['biases'])
    #activation function rectify linear
    l1 = tf.nn.relu(l1)
    
    l2= tf.add(tf.matmul(l1, hidden_2_layer['weights']) , hidden_2_layer['biases'])
    #activation function rectify linear
    l2 = tf.nn.relu(l2)
    
    l3= tf.add(tf.matmul(l2, hidden_3_layer['weights']) , hidden_3_layer['biases'])
    #activation function rectify linear
    l3 = tf.nn.relu(l3)
    
    output= tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    #activation function rectify linear
    return output
    
def train_neural_network(x, y):
    prediction = neural_network_model(x)
    
    cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= prediction, labels=y))
    optimizer= tf.train.AdamOptimizer().minimize(cost)
    #cycles with feed forward and backpropagation
    hm_epochs= 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss= 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y= mnist.train.next_batch(batch_size)
                _, c= sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch ',epoch, 'loss', epoch_loss)
            
        correct= tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        accuracy= tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: mnist.test.images, y:mnist.test.labels}))
        writer = tf.summary.FileWriter('./tensorboardtut', sess.graph)
        
train_neural_network(x, y)


Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch  0 loss 697081.3300857544
Epoch  1 loss 140165.29830169678
Epoch  2 loss 80144.92080688477
Epoch  3 loss 51165.835147857666
Epoch  4 loss 33245.29755613208
Epoch  5 loss 22714.187198651023
Epoch  6 loss 15290.801186463086
Epoch  7 loss 10433.671786204392
Epoch  8 loss 7843.539888284369
Epoch  9 loss 6088.5875274225355
Accuracy: 0.9445


NameError: name 'plt' is not defined

In [ ]:
#Positive Negative Sentiments
import nltk
#nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

lemmatizer= WordNetLemmatizer()
hm_lines= 1000000

def create_lexicon(pos, neg):
    lexicon= []
    for fi in[pos, neg]:
        with open(fi, 'r') as f:
            contents= f.readlines()
            for l in contents[:hm_lines]:
                all_words= word_tokenize(l.lower())
                lexicon += list(all_words)
                
    lexicon= [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts= Counter(lexicon)
    #print(w_counts)
    l2 = []
    for w in w_counts:
        if 1000 > w_counts[w] >50:
            l2.append(w)
    print(len(l2))        
    return l2

def sample_handling(sample, lexicon, classification):
    featureset=[]
    
    with open(sample, 'r') as f:
        contents= f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_word = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value= lexicon.index(word.lower())
                    features[index_value] +=1
            features= list(features)
            featureset.append([features, classification])
            
    return featureset

def create_feature_sets_and_labels(pos, neg, test_size=0.1):
    lexicon= create_lexicon(pos, neg)
    features=[]
    features+= sample_handling('pos.txt', lexicon, [1,0])
    features+= sample_handling('neg.txt', lexicon, [0,1])
    random.shuffle(features)
    
    features= np.array(features)
    
    testing_size= int(test_size * len(features))
    
    train_x= list(features[:, 0][: -testing_size])
    train_y= list(features[:, 1][: -testing_size])
    
    test_x= list(features[:, 0][ -testing_size: 0])
    test_y= list(features[:, 1][ -testing_size: 0])
    
    return train_x, train_y, test_x, test_y

if __name__=='__main__':
    train_x, train_y, test_x, test_y= create_feature_sets_and_labels('pos.txt', 'neg.txt')
    
    with open('sentiment_set.pickle', 'wb') as f:
        pickle.dump([train_x, train_y, test_x, test_y], f)
    #for i in range(5):
    #    print(train_x[i], train_y[i], test_x[i], test_y[i])
    

In [ ]:
import tensorflow as tf
import numpy as np


n_nodes_hl1= 250
n_nodes_hl2= 250
n_nodes_hl3= 100

n_classes=2
batch_size=200
#height x width
x=tf.placeholder('float', [None, len(train_x[0])])
y=tf.placeholder('float')

def neural_network_model(data):
    
    hidden_1_layer= {'weights': tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    
    output_layer= {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1= tf.add(tf.matmul(data, hidden_1_layer['weights']) , hidden_1_layer['biases'])
    #activation function rectify linear
    l1 = tf.nn.relu(l1)
    
    l2= tf.add(tf.matmul(l1, hidden_2_layer['weights']) , hidden_2_layer['biases'])
    #activation function rectify linear
    l2 = tf.nn.relu(l2)
    
    l3= tf.add(tf.matmul(l2, hidden_3_layer['weights']) , hidden_3_layer['biases'])
    #activation function rectify linear
    l3 = tf.nn.relu(l3)
    
    output= tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    #activation function rectify linear
    return output
    
def train_neural_network(x, y):
    prediction = neural_network_model(x)
    
    cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= prediction, labels=y))
    optimizer= tf.train.AdamOptimizer().minimize(cost)
    #cycles with feed forward and backpropagation
    hm_epochs= 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss= 0
            i=0
            while i < len(train_x):
                start= i
                end = i + batch_size
                epoch_x= np.array(train_x[start:end])
                epoch_y= np.array(train_y[start:end])
                _, c= sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
                i+= batch_size
            print('Epoch ',epoch, 'loss', epoch_loss)
            
        correct= tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        accuracy= tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_x, y: test_y}))
        
train_neural_network(x, y)
#plt.imgshow(mnist.test.images[0])